This file is used to extract scripts and labels from dataset IEMOCAP.
1. Extract emotion label, save(start_time, end_time, sentence audio file name, emotion label) in df_iemocap_label.csv file
2. Extract script , store (start_time, end_time, audio file name of the sentence, script)in df_iemocap_script.csv file
3. Merge two data file, 
   change column "emotion" to "label"
   add a new column "index" to store the index of current script in current conservation. 
   add a new column "next_label" to store the future sentence emotion. for the last sentence of current conservation. set as "None"
   save to df_imeocap.csv file
  

# Extract Emotion Label

Test for one file first

In [2]:
import re

# first test with one file
#file_path = '/kaggle/input/iemocapfullrelease/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro01.txt'
file_path = '../../dataset/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro01.txt'
useful_regex = re.compile(r'\[.+\]\n', re.IGNORECASE)
with open(file_path) as f:
    file_content = f.read()
    
info_lines = re.findall(useful_regex, file_content)
for l in info_lines[0:10]:
    print(l.strip().split('\t'))

['[START_TIME - END_TIME] TURN_NAME EMOTION [V, A, D]']
['[6.2901 - 8.2357]', 'Ses01F_impro01_F000', 'neu', '[2.5000, 2.5000, 2.5000]']
['[10.0100 - 11.3925]', 'Ses01F_impro01_F001', 'neu', '[2.5000, 2.5000, 2.5000]']
['[14.8872 - 18.0175]', 'Ses01F_impro01_F002', 'neu', '[2.5000, 2.5000, 2.5000]']
['[19.2900 - 20.7875]', 'Ses01F_impro01_F003', 'xxx', '[2.5000, 3.0000, 3.0000]']
['[21.3257 - 24.7400]', 'Ses01F_impro01_F004', 'xxx', '[2.5000, 3.0000, 2.5000]']
['[27.4600 - 31.4900]', 'Ses01F_impro01_F005', 'neu', '[2.5000, 3.5000, 2.0000]']
['[38.9650 - 43.5900]', 'Ses01F_impro01_F006', 'fru', '[2.0000, 3.5000, 3.5000]']
['[46.5800 - 52.1900]', 'Ses01F_impro01_F007', 'fru', '[2.5000, 3.5000, 3.5000]']
['[56.1600 - 58.8225]', 'Ses01F_impro01_F008', 'fru', '[2.0000, 3.5000, 3.5000]']


# Extract all emotion label to one file

In [3]:
import os

info_line = re.compile(r'\[.+\]\n', re.IGNORECASE)
#start_times, end_times, wav_file_names, emotions, vals, acts, doms = [], [], [], [], [], [], []
start_times, end_times, wav_file_names, emotions = [], [], [], []
for sess in range(1, 6):
    #emo_evaluation_dir = '/kaggle/input/iemocapfullrelease/IEMOCAP_full_release/Session{}/dialog/EmoEvaluation/'.format(sess)    
    emo_evaluation_dir = '../../dataset/IEMOCAP_full_release/Session{}/dialog/EmoEvaluation/'.format(sess)
    evaluation_files = [l for l in os.listdir(emo_evaluation_dir) if 'Ses' in l]
    for file in evaluation_files:
        with open(emo_evaluation_dir + file) as f:
            content = f.read()
        info_lines = re.findall(info_line, content)
        for line in info_lines[1:]:  # the first line is a header
            start_end_time, wav_file_name, emotion, val_act_dom = line.strip().split('\t')
            start_time, end_time = start_end_time[1:-1].split('-')
            # val, act, dom = val_act_dom[1:-1].split(',')
            # val, act, dom = float(val), float(act), float(dom)
            start_time, end_time = float(start_time), float(end_time)
            start_times.append(start_time)
            end_times.append(end_time)
            wav_file_names.append(wav_file_name)
            emotions.append(emotion)
            # vals.append(val)
            # acts.append(act)
            # doms.append(dom)

In [4]:
import pandas as pd

df_iemocap_label = pd.DataFrame(columns=['start_time', 'end_time', 'wav_file', 'emotion'])
#df_iemocap_label = pd.DataFrame(columns=['start_time', 'end_time', 'wav_file', 'emotion', 'val', 'act', 'dom'])

df_iemocap_label['start_time'] = start_times
df_iemocap_label['end_time'] = end_times
df_iemocap_label['wav_file'] = wav_file_names
df_iemocap_label['emotion'] = emotions
# df_iemocap_label['val'] = vals
# df_iemocap_label['act'] = acts
# df_iemocap_label['dom'] = doms

df_iemocap_label.head()
df_iemocap_label.tail()

,start_time,end_time,wav_file,emotion
10034,236.57,244.83,Ses05M_script03_2_M041,ang
10035,244.84,246.58,Ses05M_script03_2_M042,ang
10036,246.59,248.83,Ses05M_script03_2_M043,ang
10037,255.86,260.33,Ses05M_script03_2_M044,ang
10038,260.34,266.35,Ses05M_script03_2_M045,ang


In [11]:
# export to csv file
df_iemocap_label.to_csv('../../data/df_iemocap_label.csv', index=False)

# Extract Transcript

test on single file

In [7]:
import re

# first test with one file
file_path = '../../dataset/IEMOCAP_full_release/Session1/dialog/transcriptions/Ses01F_impro01.txt'

#regex = re.compile(r"^(\S+)\s+\[([\d.-]+)\]:\s*(.*)$")            # divide to filename, time_stamp, script
regex = re.compile(r"^(\S+)\s+\[([\d.]+)-([\d.]+)\]:\s*(.*)$")     # divide to filename, start_time, end_time, script
                   
    
with open(file_path) as f:
    file_content = f.read()   
    lines = file_content.split('\n')
    for line in lines:   
        match = regex.match(line.strip())
        if match:      
            parts = line.strip().split("\t")        
            filename = match.group(1)
            start_time = match.group(2)
            end_time = match.group(3)     
            script = match.group(4)       
            print(filename,start_time,end_time,script)
           


Ses01F_impro01_F000 006.2901 008.2357 Excuse me.
Ses01F_impro01_M000 007.5712 010.4750 Do you have your forms?
Ses01F_impro01_F001 010.0100 011.3925 Yeah.
Ses01F_impro01_M001 010.9266 014.6649 Let me see them.
Ses01F_impro01_F002 014.8872 018.0175 Is there a problem?
Ses01F_impro01_M002 016.8352 019.7175 Who told you to get in this line?
Ses01F_impro01_F003 019.2900 020.7875 You did.
Ses01F_impro01_F004 021.3257 024.7400 You were standing at the beginning and you directed me.
Ses01F_impro01_M003 023.4700 028.0300 Okay. But I didn't tell you to get in this line if you are filling out this particular form.
Ses01F_impro01_F005 027.4600 031.4900 Well what's the problem?  Let me change it.
Ses01F_impro01_M004 028.3950 031.2117 This form is a Z.X.four.
Ses01F_impro01_M005 031.2660 039.3875 You can't--  This is not the line for Z.X.four.  If you're going to fill out the Z.X.four, you need to have a different form of ID.
Ses01F_impro01_F006 038.9650 043.5900 What?  I'm getting an ID.  This is 

# Extract all the transcript to one file

In [8]:
import os
import re

start_times, end_times, wav_file_names, scripts = [], [], [], []
regex = re.compile(r"^(\S+)\s+\[([\d.]+)-([\d.]+)\]:\s*(.*)$") 

for sess in range(1, 6):  
    transcript_path = os.path.join("../../dataset/IEMOCAP_full_release", f"Session{sess}/dialog/transcriptions")
    #audio_path = os.path.join("/kaggle/input/iemocapfullrelease/IEMOCAP_full_release", f"Session{sess}/sentences/wav")   
    
    for file in os.listdir(transcript_path):
        if file.endswith(".txt"):
            with open(os.path.join(transcript_path, file), "r") as f:
                file_content = f.read()   
                lines = file_content.split('\n')
                for line in lines:   
                    match = regex.match(line.strip())
                    if match:      
                        parts = line.strip().split("\t")  
                        wav_file_name = match.group(1)
                        start_time = match.group(2)
                        end_time = match.group(3)     
                        script = match.group(4)       
                        start_time, end_time = float(start_time), float(end_time)
                        start_times.append(start_time)
                        end_times.append(end_time)                        
                        wav_file_names.append(wav_file_name)
                        scripts.append(script)

In [9]:

df_iemocap_script = pd.DataFrame(columns=['start_time', 'end_time', 'wav_file', 'script'])

df_iemocap_script ['start_time'] = start_times
df_iemocap_script ['end_time'] = end_times
df_iemocap_script ['wav_file'] = wav_file_names
df_iemocap_script ['script'] = scripts

df_iemocap_script .head()
df_iemocap_script .tail()

,start_time,end_time,wav_file,script
10082,255.86,260.33,Ses05M_script03_2_M044,oh! Marry you again? I wouldn't marry you agai...
10083,259.45,260.41,Ses05M_script03_2_F042,Beast
10084,260.34,266.35,Ses05M_script03_2_M045,You're a wicked little vampire. And I pray to...
10085,260.42,261.43,Ses05M_script03_2_F043,Brute
10086,261.91,263.07,Ses05M_script03_2_F044,pig


In [10]:
# export to csv file
df_iemocap_script.to_csv('../../data/df_iemocap_script.csv', index=False)

Merge label and script to one file used for emotion recognition (label reprents emotion of  current uttence and audio )

In [ ]:

#using inner merge to only keep rows that have the same start_time, end_time, wav_file values.
df_imeocap = pd.merge(df_iemocap_script, df_iemocap_label, on=['start_time', 'end_time', 'wav_file'], how='inner')
df_imeocap.head()
df_imeocap.tail()



,start_time,end_time,wav_file,script,emotion
10028,255.86,260.33,Ses05M_script03_2_M044,oh! Marry you again? I wouldn't marry you agai...,ang
10029,259.45,260.41,Ses05M_script03_2_F042,Beast,ang
10030,260.34,266.35,Ses05M_script03_2_M045,You're a wicked little vampire. And I pray to...,ang
10031,260.42,261.43,Ses05M_script03_2_F043,Brute,ang
10032,261.91,263.07,Ses05M_script03_2_F044,pig,ang


In [ ]:
# add a new column to show the index of current script in current conversition.

# Initialize the index column
df_imeocap["index"] = 0

# Iterate over the DataFrame and update the index column
for i in range(len(df_imeocap)):
    if (df_imeocap.loc[i, "wav_file"].endswith("000") and df_imeocap.loc[i+1, "wav_file"].endswith("000")):  
        df_imeocap.loc[i, "index"] = 1
    else:
        df_imeocap.loc[i, "index"] = df_imeocap.loc[i - 1, "index"] + 1 if i > 0 else 1  # Ensure the first row is handled properly
#df_imeocap.head()
#df_imeocap.tail()

,start_time,end_time,wav_file,script,emotion,index
0,6.2901,8.2357,Ses01F_impro01_F000,Excuse me.,neu,1
1,7.5712,10.4750,Ses01F_impro01_M000,Do you have your forms?,fru,2
2,10.0100,11.3925,Ses01F_impro01_F001,Yeah.,neu,3
3,10.9266,14.6649,Ses01F_impro01_M001,Let me see them.,fru,4
4,14.8872,18.0175,Ses01F_impro01_F002,Is there a problem?,neu,5


In [ ]:

# Initialize the index column
df_imeocap["next_label"] = ""

# Iterate over the DataFrame and update the label column
for i in range(len(df_imeocap)-1):   
    if (not df_imeocap.loc[i, "wav_file"].endswith("000") and df_imeocap.loc[i+1, "wav_file"].endswith("000")):  
        df_imeocap.loc[i, "next_label"] = "None"
    else:
        df_imeocap.loc[i, "next_label"] = df_imeocap.loc[i + 1, "emotion"]  
df_imeocap.loc[len(df_imeocap)-1, "next_label"] = "None"


In [ ]:
df_imeocap.head()

,start_time,end_time,wav_file,script,emotion,index,next_label
0,6.2901,8.2357,Ses01F_impro01_F000,Excuse me.,neu,1,fru
1,7.5712,10.4750,Ses01F_impro01_M000,Do you have your forms?,fru,2,neu
2,10.0100,11.3925,Ses01F_impro01_F001,Yeah.,neu,3,fru
3,10.9266,14.6649,Ses01F_impro01_M001,Let me see them.,fru,4,neu
4,14.8872,18.0175,Ses01F_impro01_F002,Is there a problem?,neu,5,fru


In [ ]:
df_imeocap.tail()

,start_time,end_time,wav_file,script,emotion,index,next_label
10028,255.86,260.33,Ses05M_script03_2_M044,oh! Marry you again? I wouldn't marry you agai...,ang,87,ang
10029,259.45,260.41,Ses05M_script03_2_F042,Beast,ang,88,ang
10030,260.34,266.35,Ses05M_script03_2_M045,You're a wicked little vampire. And I pray to...,ang,89,ang
10031,260.42,261.43,Ses05M_script03_2_F043,Brute,ang,90,ang
10032,261.91,263.07,Ses05M_script03_2_F044,pig,ang,91,None


In [ ]:
# change column "emotion" to "label"
df_imeocap = df_imeocap.rename(columns={"emotion": "label"})
# export to csv file
df_imeocap.to_csv('../../data/df_iemocap.csv', index=False)